In [22]:
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
from gesla_functions import *
import os
import sys
import glob

This code takes high-frequency tide gauge records from GESLA 3, averages them as hourly records, select for a specific year and region, smooths them by means of a lowess filter to remove tide contributions and correct them for the Dynamic Atmospheric Correction. 

The whole processing is needed to make GESLA dataset comparable to altimetry estimations. The final output of this code can be used in compare_TG_nodac.

Note that the location and naming of the original GESLA data and of the DAC correction from AVISO are hard coded.

The following files will be created by the code (naming changes depending on the choices). Files 2 to 4 are only temporary files and removed at the end of the processing

1) A file containing a selection of the original GESLA data saved in xarray format in the location gesla_output_dir
This is a long process, if needed the parameter "decision_gesla" needs to be set as 'yes'

2) gesla_2021update_northsea.nc 
A selection of the original GESLA dataset in a specific region

3) gesla_2021_selected_update_northsea
Tide gauges with no data after the 15th of April of the selected year are excluded. If two tide gauges are closer than 1.5 km, only the longest record is kept.

4) gesla_2021_selected_lowessupdate_northsea
Lowess filter is applied to remove tides

5) gesla_2021_selected_lowess_dac_correctedupdate_northsea
DAC correction is applied



In [23]:

# Decide whether 
decision_gesla = 'no'


# Box in which data will be geographically selected 
max_lat = 61.0
min_lat = 50.0
max_lon = 12.2
min_lon = -4.0 

# Additional naming added in the output products
add = 'update_northsea'

# Location in which the selected gesla data will be saved 
gesla_output_dir = '/nfs/public_ads/Oelsmann/marcello/gesla_v3/'

# Location in which the DAC correction for the tide gauge data will be saved
out_dir='/nfs/public_ads/Oelsmann/marcello/gesla_v3/'

# Years to be considered in the tide gauge dataset
years_in = [2021]

# Location where the corrected tide gauge xarray will be stored
tg_directory = '/DGFI8/H/work_marcello/machine_learning_altimetry_validation/'

In [24]:
def make_gesla(start_date = pd.to_datetime('01-01-2021'),add='update',gesla_output_dir = '/nfs/public_ads/Oelsmann/marcello/gesla_v3/',years_in = [2021]):
    dire = '/nfs/DGFI8/D/tide-gauges/GESLA_3/'
    #filenames = os.listdir(dire)
    meta = '/nfs/DGFI8/D/tide-gauges/GESLA_3_meta/GESLA3_ALL_2.csv'    
    gesla = GeslaDataset(meta,dire)
    #filenames = gesla.meta[gesla.meta['end_date_time'] > pd.to_datetime('31-12-2021')].filename.values
    filenames = gesla.meta.filename.values
    gesla_all = gesla.files_to_xarray(filenames = filenames,apply_use_flag=True,
                       start_date =start_date, resampling = 'H')
    gesla_all.to_netcdf(gesla_output_dir+'gesla_'+str(years_in[0])+add+'.nc')


def make_DAC_aviso(dataset,DAC_name='',name = 'dac_dif_26296_12.nc',dire = '/DGFI8/D/ib/DAC.AVISO/',years_in = [2021],out_dir='/nfs/public_ads/Oelsmann/marcello/gesla_v3/'):
    testfile = xr.open_dataset(dire +str(years_in[0])+'/'+ name)
    testfile=testfile.rename({'latitude':'lat','longitude':'lon'})
    lon_nw,lat_nw = np.meshgrid(testfile.lon,testfile.lat)
    start = np.full(lon_nw.shape,False)
    id1=[]
    id2=[]
    for lat,lon in zip(dataset.lat.values,dataset.lon.values):
        diff =abs((lon_nw-lon))+ abs((lat_nw-lat))
        out = np.where(diff == np.min(diff))
        id1.append(out[0][0])
        id2.append(out[1][0])
    for year in years_in:
        data  = glob.glob(dire+str(year)+'/*nc')
        DATA=np.empty((len(data),len(id1)))*np.nan
        TIME=pd.DataFrame(np.empty(len(data))*np.nan)
        i=0
        for dataname in data:
            print(i, dataname)
            dat = xr.open_dataset(dataname)

            DATA[i,:]=dat.dac.values[id1,id2]
            TIME.loc[i]=np.datetime64(dat.dac.date[:-4])     
            i=i+1
        DATA = pd.DataFrame(DATA)
        DATA['time']=TIME
        DATA.to_csv(out_dir+'data_for_time_'+str(year))

In [25]:
# Very long process!!!

if decision_gesla == 'yes' :

    make_gesla(start_date = pd.to_datetime('01-01-2021'),add='update')

In [26]:
gesla_all = xr.open_dataset(gesla_output_dir+'gesla_'+str(years_in[0])+'update.nc')
gesla_all

<xarray.Dataset>
Dimensions:                  (date_time: 6207, station: 1164)
Coordinates:
  * station                  (station) int64 0 1 2 3 4 ... 1160 1161 1162 1163
  * date_time                (date_time) datetime64[ns] 2021-01-01T01:00:00 ....
Data variables: (12/27)
    sea_level                (station, date_time) float64 ...
    qc_flag                  (station, date_time) float64 ...
    use_flag                 (station, date_time) float64 ...
    filename                 (station) object ...
    site_name                (station) object ...
    site_code                (station) object ...
    ...                       ...
    datum_information        (station) object ...
    instrument               (station) object ...
    precision                (station) object ...
    null_value               (station) float64 ...
    gauge_type               (station) object ...
    overall_record_quality   (station) object ...

In [27]:
# Limit to North Sea

gesla_all = gesla_all.where((gesla_all.latitude >= min_lat) & (gesla_all.latitude <= max_lat) & (gesla_all.longitude >= min_lon) & (gesla_all.longitude <= max_lon), drop=True) 
# Loop through variables in the dataset and put unknown data types to string, otherwise "to_netcdf" does not work
for varname in gesla_all.variables:
    if gesla_all[varname].dtype not in ['float64', 'datetime64[ns]','int64']:
        gesla_all[varname] = gesla_all[varname].astype(dtype='U80', copy=False)

filename_and_path_limit = tg_directory+'gesla_'+str(years_in[0])+add+'.nc'
gesla_all.to_netcdf(filename_and_path_limit)
os.chmod(filename_and_path_limit, 0o777)

In [28]:
#Content of select_and_dropdupl
#Note that tide gauges with no data after the 15th of April of the selected year are excluded
#If two tide gauges are closer than 1.5 km, only the longest record is kept

gesla_all_sorted = gesla_all.sortby('site_name')
limit = 1500
counts = gesla_all_sorted['sea_level'].count(dim='date_time').values
coords =np.stack([gesla_all_sorted.latitude.values,gesla_all_sorted.longitude.values])
distancs = []
for coord1 in coords.T:
    distancs.append(haversine(coord1, coords))
distancs = np.vstack(distancs)
pairs = []
reject_indices = []
for i in range(distancs.shape[0]):
    sub = distancs[:,i]
    idx = np.argwhere(   (sub<limit) & (sub!=0) ) 
    if len(idx)>0:
        pairs.append([i,idx[0]])
        if counts[i]<=counts[int(idx[0])]:
            reject_indices.append(i)
        else:
            reject_indices.append(int(idx[0]))
reject_at  = pd.DataFrame(reject_indices).drop_duplicates().values.flatten()
gesla_selected = gesla_all_sorted.sel({'station':~np.isin(np.arange(distancs.shape[0]),reject_at)})
mask = (gesla_selected['date_time'].to_dataframe()['date_time'] >  pd.to_datetime('04-15-'+str(years_in[0]))).values
counts_after_april = gesla_selected.sel({'date_time':mask}).count(dim='date_time')['sea_level']
gesla_selected_final = gesla_selected.where(counts_after_april > 1,drop=True)
gesla_selected_final.attrs={'comment':'duplicates are rejected; longest TG record of a TG pair within 1.5km is kept. TG must contain data after 04-15-. Use-flag applied. Hourly averaged.'}

filename_and_path_dropupl = tg_directory+'gesla_'+str(years_in[0])+'_selected_'+add+'.nc'
gesla_selected_final.to_netcdf(filename_and_path_dropupl)
os.chmod(filename_and_path_dropupl, 0o777)

In [29]:
#apply_lowess_filter
import copy
lowess = sm.nonparametric.lowess
gesla_selected_final=xr.open_dataset(tg_directory+'gesla_'+str(years_in[0])+'_selected_'+add+'.nc')
gesla_selected_final['sea_level_lowess']=copy.deepcopy(gesla_selected_final['sea_level'])
frac = 40/len(gesla_selected_final.date_time)
x=np.arange(len(gesla_selected_final.date_time))
for i in range(len(gesla_selected_final.station)):
    y = gesla_selected_final['sea_level'][i,:].values
    w = lowess(y, x, frac=frac)
    gesla_selected_final['sea_level_lowess'][i,np.isin(x,w[:,0])] = w[:,1]
    print(i)
    
filename_and_path_lowess = tg_directory+'gesla_'+str(years_in[0])+'_selected_lowess'+add+'.nc'
gesla_selected_final.to_netcdf(filename_and_path_lowess)
os.chmod(filename_and_path_lowess, 0o777)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [30]:
#compute_dac_gesla
gesla = xr.open_dataset(tg_directory+'gesla_'+str(years_in[0])+'_selected_lowess'+add+'.nc')
dataset= gesla
dataset = dataset.rename({'latitude':'lat','longitude':'lon'})
lon= dataset['lon']
lon[lon <0 ] = lon[lon <0 ]+360
dataset['lon'][:] = lon
make_DAC_aviso(dataset)

0 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26267_18.nc
1 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26021_12.nc
2 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25991_18.nc
3 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26228_18.nc
4 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26240_06.nc
5 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25979_12.nc
6 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25934_00.nc
7 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26217_18.nc
8 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26231_18.nc
9 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26263_12.nc
10 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26182_00.nc
11 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26166_12.nc
12 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26242_00.nc
13 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26233_00.nc
14 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25953_18.nc
15 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25924_00.nc
16 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26080_06.nc
17 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26295_06.nc
18 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26191_06.nc
19 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25923_06.nc
20 /DGFI8/

164 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26093_12.nc
165 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26272_00.nc
166 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26216_18.nc
167 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26138_18.nc
168 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25987_12.nc
169 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26102_00.nc
170 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26139_12.nc
171 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25946_06.nc
172 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26166_06.nc
173 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25964_12.nc
174 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25942_12.nc
175 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26290_12.nc
176 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26017_18.nc
177 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25994_18.nc
178 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25914_06.nc
179 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26006_00.nc
180 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26120_00.nc
181 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26066_06.nc
182 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26055_00.nc
183 /DGFI8/D/ib/DAC.AVISO/2021/

330 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26005_06.nc
331 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26052_12.nc
332 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26188_06.nc
333 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26071_12.nc
334 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26085_12.nc
335 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26155_18.nc
336 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26280_12.nc
337 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26045_18.nc
338 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26197_06.nc
339 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25939_18.nc
340 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26227_00.nc
341 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26070_18.nc
342 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25954_06.nc
343 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26011_18.nc
344 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25989_12.nc
345 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25957_06.nc
346 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26256_00.nc
347 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26129_18.nc
348 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25989_06.nc
349 /DGFI8/D/ib/DAC.AVISO/2021/

495 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26077_06.nc
496 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26257_12.nc
497 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25948_18.nc
498 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26246_00.nc
499 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26110_12.nc
500 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25997_06.nc
501 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26107_00.nc
502 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26158_12.nc
503 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26030_00.nc
504 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26176_12.nc
505 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26211_18.nc
506 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26013_06.nc
507 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25944_12.nc
508 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25916_06.nc
509 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25952_12.nc
510 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25973_00.nc
511 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26113_00.nc
512 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26102_18.nc
513 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26261_06.nc
514 /DGFI8/D/ib/DAC.AVISO/2021/

661 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26268_00.nc
662 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25940_12.nc
663 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26254_18.nc
664 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26182_12.nc
665 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26065_18.nc
666 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26239_06.nc
667 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26141_06.nc
668 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26002_00.nc
669 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26114_12.nc
670 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26197_00.nc
671 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26117_06.nc
672 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26064_12.nc
673 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25932_06.nc
674 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26030_18.nc
675 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26174_18.nc
676 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25918_06.nc
677 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26148_06.nc
678 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26142_06.nc
679 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26037_12.nc
680 /DGFI8/D/ib/DAC.AVISO/2021/

829 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26022_00.nc
830 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26131_12.nc
831 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26187_18.nc
832 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25995_18.nc
833 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26152_06.nc
834 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26263_18.nc
835 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25996_12.nc
836 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26010_12.nc
837 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26119_12.nc
838 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26111_18.nc
839 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26202_06.nc
840 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26080_00.nc
841 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26114_18.nc
842 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26029_18.nc
843 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25992_18.nc
844 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26125_06.nc
845 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26177_18.nc
846 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26117_18.nc
847 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26006_18.nc
848 /DGFI8/D/ib/DAC.AVISO/2021/

995 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26228_12.nc
996 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25980_18.nc
997 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26210_18.nc
998 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26088_18.nc
999 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25978_18.nc
1000 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25961_00.nc
1001 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26135_06.nc
1002 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26215_12.nc
1003 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25964_18.nc
1004 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26024_18.nc
1005 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26031_12.nc
1006 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26165_12.nc
1007 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26275_00.nc
1008 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26295_00.nc
1009 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26217_06.nc
1010 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26050_06.nc
1011 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25988_06.nc
1012 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25971_18.nc
1013 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25944_00.nc
1014 /DGFI8/D/ib/

1156 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26110_00.nc
1157 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25983_12.nc
1158 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26162_12.nc
1159 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25964_00.nc
1160 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26005_12.nc
1161 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26205_06.nc
1162 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26239_00.nc
1163 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26227_06.nc
1164 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26040_06.nc
1165 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26043_18.nc
1166 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26264_06.nc
1167 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26216_00.nc
1168 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26211_06.nc
1169 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26241_06.nc
1170 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26193_12.nc
1171 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26196_06.nc
1172 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26159_00.nc
1173 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26287_18.nc
1174 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26135_00.nc
1175 /DGFI8/

1315 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26280_00.nc
1316 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25943_00.nc
1317 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25977_12.nc
1318 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26089_06.nc
1319 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26018_06.nc
1320 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26009_00.nc
1321 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26271_18.nc
1322 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26118_00.nc
1323 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26155_12.nc
1324 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26011_12.nc
1325 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26079_06.nc
1326 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25948_00.nc
1327 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26125_18.nc
1328 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26294_06.nc
1329 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25951_18.nc
1330 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26194_18.nc
1331 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26151_12.nc
1332 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26024_00.nc
1333 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26206_06.nc
1334 /DGFI8/

1477 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26128_12.nc
1478 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26126_12.nc
1479 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26004_00.nc
1480 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26042_12.nc
1481 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26048_00.nc
1482 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25938_18.nc
1483 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25996_00.nc
1484 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26090_06.nc
1485 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26286_00.nc
1486 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25948_12.nc
1487 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26079_12.nc
1488 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26163_00.nc
1489 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26042_06.nc
1490 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26036_06.nc
1491 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26244_06.nc
1492 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_25930_12.nc
1493 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26204_00.nc
1494 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26048_12.nc
1495 /DGFI8/D/ib/DAC.AVISO/2021/dac_dif_26223_06.nc
1496 /DGFI8/

In [31]:
# Here I first need to apply lowess!!!

#correct_gesla_for_dac()

dire = '/DGFI8/D/ib/DAC.AVISO/'


gesla = xr.open_dataset(tg_directory+'gesla_'+str(years_in[0])+'_selected_lowess'+add+'.nc')
dac = pd.read_csv(out_dir+'data_for_time_'+str(years_in[0])).sort_values(by='time')
dac=dac.set_index('time')
dac.index = pd.to_datetime(dac.index)
dac_sub = dac[(pd.DatetimeIndex(dac.index.values) >= pd.Timestamp(gesla['date_time'][0].values)) & (pd.DatetimeIndex(dac.index.values) <= pd.Timestamp(gesla['date_time'][-1].values))]
gesla_time = gesla.loc[dict(date_time=slice(dac.index[0], dac.index[-1]))]
gesla_time['dac'] = copy.deepcopy(gesla_time['sea_level_lowess'])*np.nan
dac_res = (dac_sub.resample('1H').mean()).interpolate(method='cubic',limit=10)
gesla_time['dac'][:,np.isin(gesla_time.date_time,dac_res.index)] = dac_res.iloc[:,1:].values.T
gesla_time['sla_dac']=gesla_time['sea_level_lowess']-gesla_time['dac']
gesla_time['sla_dac_no_loess']=gesla_time['sea_level']-gesla_time['dac']

filename_and_path = tg_directory+'gesla_'+str(years_in[0])+'_selected_lowess_dac_corrected'+add+'.nc'
gesla_time.to_netcdf(filename_and_path)
os.chmod(filename_and_path, 0o777)

In [32]:
os.remove(filename_and_path_limit)
os.remove(filename_and_path_dropupl)
os.remove(filename_and_path_lowess)